# Per Bounding Box Luminosity Calculation

This notebook demonstrates how to calculate the luminosity of images and their respective bounding boxes. We will then write these calculated luminosity properties to a 3LC Run.

In [ ]:
PROJECT_NAME = "Luminosity"
DATASET_NAME = "COCO128"
INSTALL_DEPENDENCIES = False
TEST_DATA_PATH = "../tests/test_data/data"
TLC_PUBLIC_EXAMPLES_DEVELOPER_MODE = True

In [ ]:
if INSTALL_DEPENDENCIES:
    %pip --quiet install ipykernel ipywidgets
    %pip --quiet install torch --index-url https://download.pytorch.org/whl/cu118
    %pip --quiet install torchvision --index-url https://download.pytorch.org/whl/cu118
    %pip --quiet install tlc

In [ ]:
!3lc config --list

## Import and Initialize

First, let's import necessary modules and initialize a 3LC Run using a input table URL.

In [ ]:
from __future__ import annotations

from io import BytesIO

import tqdm
import numpy as np
from PIL import Image

import tlc

In [ ]:
### HIDDEN CELL ###

if not TLC_PUBLIC_EXAMPLES_DEVELOPER_MODE:
    from tlc.client.utils import (
        TLC_PUBLIC_EXAMPLES_RUN_ROOT,
        TLC_PUBLIC_EXAMPLES_TABLE_ROOT,
        TLC_PUBLIC_EXAMPLES_COCO_128_DATA_ALIAS_NAME,
    )

    from tlc.core.objects.mutable_objects import Configuration
    from tlc.core.objects.tables.system_tables.indexing_tables import TableIndexingTable
    from tlc import Url, UrlAliasRegistry

    print(f"Runs and Tables will be written to remote location: '{TLC_PUBLIC_EXAMPLES_RUN_ROOT}' and '{TLC_PUBLIC_EXAMPLES_TABLE_ROOT}'")
    Configuration.instance().run_root_url = TLC_PUBLIC_EXAMPLES_RUN_ROOT
    Configuration.instance().table_root_url = TLC_PUBLIC_EXAMPLES_TABLE_ROOT

    local_data_location = tlc.Url(TEST_DATA_PATH + "/coco128").to_absolute()

    # Register COCO128 data alias
    if TLC_PUBLIC_EXAMPLES_COCO_128_DATA_ALIAS_NAME not in UrlAliasRegistry.instance()._url_aliases:
        UrlAliasRegistry.instance().register_url_alias(
            TLC_PUBLIC_EXAMPLES_COCO_128_DATA_ALIAS_NAME,
            local_data_location.to_str(),
        )
        print(f"Local data at '{local_data_location}' hidden under alias '{TLC_PUBLIC_EXAMPLES_COCO_128_DATA_ALIAS_NAME}'")

    TableIndexingTable.instance().scan_urls.append(Url(TLC_PUBLIC_EXAMPLES_TABLE_ROOT))

## Set Up Input Table

We will use a `TableFromCoco` to load the input dataset from a annotations file and a folder of images.

In [ ]:
table_url = tlc.Table.default_write_location() / f"{PROJECT_NAME}/table_from_coco.json"

annotations_file = tlc.Url(TEST_DATA_PATH  + "/coco128/annotations.json").to_absolute()
images_dir = tlc.Url(TEST_DATA_PATH + "/coco128/images").to_absolute()

table_from_coco = tlc.TableFromCoco(
    url=table_url,
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    input_url=annotations_file.to_relative(),
    image_folder_url=images_dir.to_relative(),
    row_cache_url="../table_from_coco.parquet",
)

print(table_from_coco.url)

In [ ]:
run = tlc.init(project_name=PROJECT_NAME)
run.add_input_table(table_from_coco)

## Calculate the Luminoisty of Images and Bounding Boxes

In this section, we will calculate the luminosity property for each image as well as for each bounding box within the images.

We build the variables `per_image_luminosity` and `per_bb_luminosity` to store the luminosity properties for each image and bounding box, respectively.

In [ ]:
def calculate_luminosity(image: Image) -> float:
    np_image = np.array(image)
    axes_to_reduce = tuple(range(np_image.ndim - 1))
    avg_luminosity = np.mean(np_image, axis=axes_to_reduce) / 255.0
    return float(np.mean(avg_luminosity))

In [ ]:
per_bb_luminosity: list[list[float]] = []
per_image_luminosity: list[float] = []

bb_schema = table_from_coco.schema.values["rows"].values["bbs"].values["bb_list"]

for row in tqdm.tqdm(table_from_coco.table_rows, total=len(table_from_coco), desc="Calculating luminosity"):
    image_filename = row["image"]
    image_bbs = row["bbs"]["bb_list"]

    image_bytes = tlc.Url(image_filename).read()
    image = Image.open(BytesIO(image_bytes))

    image_luminosity = calculate_luminosity(image)
    per_image_luminosity.append(image_luminosity)

    bb_luminosity_list: list[float] = []
    h, w = image.size

    for bb in image_bbs:
        bb_crop = tlc.BBCropInterface.crop(image, bb, bb_schema)
        bb_luminosity = calculate_luminosity(bb_crop)
        bb_luminosity_list.append(bb_luminosity)

    per_bb_luminosity.append(bb_luminosity_list)

## Add Metrics to Run

After calculating the luminosity, we will extend the existing run with the new metrics, which will be written as a new metrics table.

In [ ]:
# Column-wise metrics
data = {
    "per_image_luminosity": per_image_luminosity,
    "per_bb_luminosity": per_bb_luminosity,
}

# Each entry in the list is a list of luminosity values for each bounding box in the image 
float_list_schema = tlc.Schema(
    value=tlc.Float32Value(
        value_min=0,
        value_max=1,
        number_role=tlc.NUMBER_ROLE_FRACTION,
    ),
    size0=tlc.DimensionNumericValue(value_min=0, value_max=1000),  # Max 1000 bounding boxes
)

# We don't need to explicitly override the per_image_luminosity column schema because is is a simple float
schemas = {
    "per_bb_luminosity": float_list_schema,
}

run.add_metrics_data(
    data,
    override_column_schemas=schemas,
    input_table_url=table_from_coco.url,
)

## Add More Metrics Based on Existing Ones

If necessary, you can also add additional metrics based on the previously calculated ones.

In [ ]:
# Fetch the metrics table we just created
metrics_table = run.metrics_tables[0]

# Find the URL of the input table used for generating this metrics table (it should be the same as the input table we used)
table_url = metrics_table.get_foreign_table_url()

In [ ]:
print(table_url)

In [ ]:
# WIP: add a column to the metrics table that is the average of the per_bb_luminosity column

avg_bb_luminosity = []

for row in metrics_table.table_rows:
    per_bb_luminosity = row["per_bb_luminosity"]
    avg_bb_luminosity.append(np.mean(per_bb_luminosity or [0]))

new_metrics_table = metrics_table.add_column(
    "avg_bb_luminosity",
    avg_bb_luminosity,
    url=metrics_table.absolute_url_from_relative(tlc.Url("../edited_metric.json")),
)
new_metrics_table.write_to_url()

run.update_metrics_table_urls({metrics_table.url: new_metrics_table.url})
new_metrics_table.write_to_url()
